# An introduction to Embeddings from Language Models (ELMo)

This presentation is inspired by the illustrations and contents from [Mihail Eric](https://www.mihaileric.com/), [Jay Alammar](http://jalammar.github.io/) and [Minsuk Heo](https://github.com/minsuk-heo).

The original paper published by AllenAI and UW researchers: [Deep Contextualized Word Representation](https://arxiv.org/pdf/1802.05365.pdf)

## Introduction

* Pretrained word embeddings have been used for downstream NLP tasks for years
* Word2Vec and GloVe are two classic examples that provide embeddings for thousands of English and other language words
* They consider context while encoding words to numerical vectors, but that's not enough
* Consider the word 'stick'. It has different meanings, depending on the context.

**Word2Vec and GloVe only have one embedding for the word 'stick' regardless of context. Not the case with ELMo:**

<img src="http://jalammar.github.io/images/elmo-embedding-robin-williams.png" alt="stick embedding" width="60%"/>

Another example:
"I read a book" vs. "I read a book, yesterday". Even though the spelling is identical, the pronanciation is different and the embedding should be as well.

* ELMo considers the context while generating an embedding for a word, hence 'Deep Contextualized'
* ELMo is a deep bidirectional language model (biLM), pretrained on a large corpus
* Layers are based on Long Short-Term Memory networks (LSTM) stacked on top of each other, working in a bidirectional manner

### Language Model Definition

Let's get our definitions straight. We first define a forward language model, then a backward language model and finally the bidirectional language model.

**Forward LM:** Given a sequence of N words, (t<sub>1</sub>, t<sub>2</sub>, ...,t<sub>N</sub>) a forward LM models the sequence by computing the probability of the t<sub>k</sub> token (word) given the history (t<sub>1</sub>, t<sub>2</sub>, ..., t<sub>k-1</sub>):

<a href="https://www.codecogs.com/eqnedit.php?latex=p(t_{1},&space;t_{2},&space;...,&space;t_{N})&space;=&space;\prod_{k=1}^{N}&space;p(t_{k}&space;|&space;t_{1},&space;t_{2},&space;...,&space;t_{k-1})" target="_blank"><img src="https://latex.codecogs.com/gif.latex?p(t_{1},&space;t_{2},&space;...,&space;t_{N})&space;=&space;\prod_{k=1}^{N}&space;p(t_{k}&space;|&space;t_{1},&space;t_{2},&space;...,&space;t_{k-1})" title="p(t_{1}, t_{2}, ..., t_{N}) = \prod_{k=1}^{N} p(t_{k} | t_{1}, t_{2}, ..., t_{k-1})" /></a>


**Backward LM:** Similar to the forward model, except that we have the words coming after the word that we want to predict its probability.

<a href="https://www.codecogs.com/eqnedit.php?latex=p(t_{1},&space;t_{2},&space;...,&space;t_{N})&space;=&space;\prod_{k=1}^{N}&space;p(t_{k}&space;|&space;t_{k&plus;1},&space;t_{k&plus;2},&space;...,&space;t_{N})" target="_blank"><img src="https://latex.codecogs.com/gif.latex?p(t_{1},&space;t_{2},&space;...,&space;t_{N})&space;=&space;\prod_{k=1}^{N}&space;p(t_{k}&space;|&space;t_{k&plus;1},&space;t_{k&plus;2},&space;...,&space;t_{N})" title="p(t_{1}, t_{2}, ..., t_{N}) = \prod_{k=1}^{N} p(t_{k} | t_{k+1}, t_{k+2}, ..., t_{N})" /></a>


**Bidirectional LM:** Combines both forward and backward model.

<a href="https://www.codecogs.com/eqnedit.php?latex=\sum_{k=1}^{N}&space;(log&space;\&space;p(t_{k}&space;|&space;t_{1},&space;t_{2},&space;...,&space;t_{k&space;-&space;1})&space;&plus;&space;log&space;\&space;p(t_{k}&space;|&space;t_{k&plus;1},&space;t_{k&plus;2},&space;...,&space;t_{N}))" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\sum_{k=1}^{N}&space;(log&space;\&space;p(t_{k}&space;|&space;t_{1},&space;t_{2},&space;...,&space;t_{k&space;-&space;1})&space;&plus;&space;log&space;\&space;p(t_{k}&space;|&space;t_{k&plus;1},&space;t_{k&plus;2},&space;...,&space;t_{N}))" title="\sum_{k=1}^{N} (log \ p(t_{k} | t_{1}, t_{2}, ..., t_{k - 1}) + log \ p(t_{k} | t_{k+1}, t_{k+2}, ..., t_{N}))" /></a>

And the goal is to maximize the above log likelihood equation by jointly maximizing the log likelihood of forward and backward directions.

**Important note:**
The main goal of language modeling is to be able to predict a word, given its surronding words. The above definitions are not specific to ELMo and every language model technique would have a similar optimization problem to solve.

## Architecture of the ELMo

ELMo is constructed by 2 layers of bidirectional LSTMs stacked up together.


<img src="https://www.mihaileric.com/static/baseline_biLM-f9173e8e65a8597d3d8a909f3aee39f1-36fb0.png" alt="ELMo structure" width="40%"/>

In the figure above, the unrolled version of ELMo is presented. The red boxes show the forward pass and the blue boxes show the backward pass.

* The forward LM tries to predict the next word given the previous words
* The backward LM tries to predict the previous word given the next words

**Important:** both forward and backward LSTMs are trained simultaneously and separately. The results of each direction **for ALL layers** are then concatenated to form the final representation of the network.

ELMo researchers also used the idea of residual connection to further enhance the performance of their model. In a [2015 paper by Microsoft researchers](https://arxiv.org/pdf/1512.03385.pdf), residual connections that add up vectors directly from the input to the middle hidden layers' weights in a deep network, helped with the performance of the model by preventing the gradient vanishing problem.

<img src="https://miro.medium.com/max/470/1*pUyst_ciesOz_LUg0HocYg.png" alt="Residual learning block" width="40%"/>

Hence, the more accurate structure of the ELMo looks like this:

<img src="https://www.mihaileric.com/static/biLM_with_residual-096e1ae8acc0d3f846f0a71da2be3449-300e1.png" alt="ELMo with Residual learning" width="40%"/>



#### Word Embedding

In the ML-based NLP tasks, we convert each word to a numerical vector before feeding it to the network. Traditionally, this is done either by an embedding layer at the first layer that learns the embeddings as the model being trained, or by using pretrained existing embeddings such as `Word2vec` or `GloVe`.


ELMo uses `character embedding` to embed the tokens. It is done by embedding the characters first and then building up upon words through permutations of characters.

It allows to capture morphological features of words (prefixes, suffixes, etc.) that might be missed using word-level embedding. Besides, the model can build up the out-of-vocabulary words by permutating its characters and hence, giving a valid embedding for such words.

Imagine the training corpus contains the words 'mark' and 'marked' but doesn't containt the word 'marking'. Upon modelling a test document that contains the word 'marking', ELMo is able to create a valid representation by building up 'marking' on 'mark' + '-ing' basis. The '-ing' could be from another word in the training corpus, like 'modelling', which was built up through the process of character embedding.

Read more about character embedding in this [2015 paper by researchers from Harvard and NYU](https://arxiv.org/pdf/1508.06615.pdf).

 The character embedding vectors then go through a `Convolution` and `Max-Pooling` layer, that allows us to build meaningful n-grams (actual English words) from character embeddings.
 
 **Remember how CNNs work very good with image processing? They are able to capture pixel features at different abstraction layers: sharp edges, color-based layers, contrast, etc. Now imagine that instead of pixels, we have characters and are trying to build up words instead of edges.**

**Time to take a look at how ELMo embed the words.**

<img src="https://www.mihaileric.com/static/modified_input_embedding-ce183e70063bae69926ef6a79e39b764-c2cda.png" alt="ELMo character embedding" width="30%"/>

**What's the deal with the `Highway` network?**

[Highway Networks was introduced in 2015 by IDSIA researchers](https://arxiv.org/pdf/1505.00387.pdf) and presents a more performant way for gradient-based training of very deep neural networks. It utilizes some 'gates' that control the flow of information through the layers of the network which smooths the process of gradient-based training of very deep networks.

In our case, ELMo utilizes character embedding as well as convolution and max-pooling for vectorizing the words. This introduces a high complexity for the training phase which could be smoothed by `highway networks`.

After this process, each token is ready to be feed to the LSTM layers.

#### ELMo representation (vector)

Now that we have the desired vector to feed the biLSTM layers, let's take a deeper look on how the final ELMo-specific vector is computed.

<img src="https://www.mihaileric.com/static/elmo_combination-a7af2b3eb2b5ceb37f3e9c5f2b066f14-6fd10.png" alt="ELMo vector computation" width="80%"/>


The ELMo representation for the k<sub>th</sub> word in the input is a weighted summation of the x<sub>k</sub> embedding, the bidirectional hidden layer weights from layer1, h<sub>1,k</sub> and layer2, h<sub>2,k</sub>.

Before jumping to the math, let's take a look at another figure that shows the structure with more details.


![ELMo detailed structure](elmo.png)


#### Jump to math

Concretely, the function `f` in the figure above does the following computation:

<a href="https://www.codecogs.com/eqnedit.php?latex=ELMo_{k}^{task}&space;=&space;\gamma&space;_{k}&space;.&space;(s_{0}^{task}.x_{k}&space;&plus;&space;s_{1}^{task}.h_{1,k}&space;&plus;&space;s_{2}^{task}.h_{2,k})" target="_blank"><img src="https://latex.codecogs.com/gif.latex?ELMo_{k}^{task}&space;=&space;\gamma&space;_{k}&space;.&space;(s_{0}^{task}.x_{k}&space;&plus;&space;s_{1}^{task}.h_{1,k}&space;&plus;&space;s_{2}^{task}.h_{2,k})" title="ELMo_{k}^{task} = \gamma _{k} . (s_{0}^{task}.x_{k} + s_{1}^{task}.h_{1,k} + s_{2}^{task}.h_{2,k})" /></a>

Where s<sub>i</sub> represents a sogtmax normalization that is applied to vectors and <span>&#947;</span><sub>k</sub> represents a task specific scaling coefficient.

Note that each downstream task (sentiment analysis, named entity recognition, question answering, etc.) requires its own training with different scaling factor coefficients and results in different ELMo embedding for each word.

To use ELMo for different tasks (transfer learning) we freeze the hidden layer weights and fine-tune the s<sub>i</sub> and <span>&#947;</span><sub>k</sub>

**Why do we include the first hidden layer weights and even the initial word embedding in the final ELMo vector computation?**

Researchers stated that the higher LSTM hidden layer helps with capturing the context while the lower LSTM hidden layer helps with capturing the syntactical features of the words. Both are necessary, so we perform a weighted summation to capture both.

The input word embedding vector also has a say in the final ELMo vector computation to further capture the embedding features before the biLSTM learning.

## Experimental results

|            Task            | Previous SOTA |  ELMo |
|:--------------------------:|:-------------:|:-----:|
| SQuAD (question/answering) |      84.4     |  85.8 |
|  SNLI (textual entailment) |      88.6     |  88.7 |
|   Semantic Role Labelling  |      81.7     |  84.6 |
|      Coref Resolution      |      67.2     |  70.4 |
|             NER            |     91.93     | 92.22 |
| SST-5 (sentiment analysis) |      53.7     |  54.7 |

ELMo achieved a better performance in all of the tasks compared to the previous state of the art model.